# Part 4: Using GPU acceleration with PyTorch

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 37.9 MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [2]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 6.8720e-01,  2.4224e-01, -1.1225e-01,  ..., -3.1547e-01,
         -6.9882e-01, -2.6660e+00],
        [-4.0570e-02,  4.6827e-01, -2.3413e-02,  ..., -6.4582e-01,
          1.2156e-01,  2.5197e+00],
        [ 1.8310e-01, -1.6466e-03,  3.0760e+00,  ...,  5.8862e-01,
          3.6877e-01, -1.7098e+00],
        ...,
        [-3.6434e-01,  5.3485e-03,  4.4073e-01,  ..., -3.6688e-01,
          3.0627e-02,  8.6036e-02],
        [ 4.0451e-01,  6.2060e-02, -3.8938e-01,  ..., -1.4642e-02,
          6.4860e-02, -8.7519e-01],
        [ 3.0766e-01,  9.0602e-01, -9.8955e-01,  ...,  3.9348e-02,
         -1.1902e+00, -2.2716e+00]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. **If you're running a non-cuda enabled version of PyTorch the following will throw an error;** if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [3]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 0.5709,  0.0595,  0.3101,  ..., -0.8325,  2.2935, -0.1034],
        [ 0.1016, -0.0428,  0.0788,  ..., -1.5525,  0.6824, -0.5954],
        [ 0.5437, -0.4461, -0.1684,  ..., -0.3316,  0.0548, -0.1346],
        ...,
        [ 0.2796,  0.0297,  0.0598,  ..., -0.1480,  1.1220, -0.0927],
        [-0.4426,  1.3755,  0.0141,  ...,  0.1255, -0.0561,  0.8649],
        [-2.3598,  0.7590, -0.0585,  ...,  0.0862, -0.0637,  1.4354]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-0.1035, -0.2530, -0.1287,  ..., -0.6224, -0.4396,  0.8815],
        [ 0.3562, -1.2855,  0.1243,  ..., -0.1678,  0.1889, -1.7594],
        [-0.2757,  0.0081, -0.4553,  ..., -0.6057,  0.0043, -0.0264],
        ...,
        [-0.1030, -0.6209,  0.8744,  ...,  0.0971, -0.3531, -1.1054],
        [ 0.3799,  0.0856, -0.0466,  ..., -0.8425,  0.1291,  0.8214],
        [-0.9357,  0.5502, -0.1919,  ...,  0.1740, -1.5871, -0.6145]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

YOUR ANSWER HERE

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

YOUR ANSWER HERE